In [ ]:
%%writefile learners_file.py

import adaptive
from functools import partial

import phase_diagram

lead_pars = dict(
    a=10, r1=50, r2=70, coverage_angle=135, angle=45, with_shell=True, which_lead=""
)

params = dict(
    alpha=20,
    B_x=0,
    B_y=0,
    B_z=0,
    Delta=110,
    g=50,
    orbital=True,
    mu_sc=100,
    c_tunnel=3 / 4,
    V_r=-50,
    mu_="lambda x0, sigma, mu_lead, mu_wire: mu_lead",
    V_="lambda z, V_0, V_r, V_l, x0, sigma, r1: 0",
    V_0=None,
    V_l=None,
    mu_lead=10,
    mu_wire=None,
    r1=None,
    sigma=None,
    x0=None,
    **phase_diagram.constants.__dict__
)


def pf(xy, params=params, lead_pars=lead_pars):
    import phase_diagram

    params["B_x"], params["mu_lead"] = xy
    lead = phase_diagram.make_lead(**lead_pars).finalized()
    return phase_diagram.calculate_pfaffian(lead, params)


def smallest_gap(xy, params=params, lead_pars=lead_pars):
    import phase_diagram

    params["B_x"], params["mu_lead"] = xy
    params = phase_diagram.parse_params(params)
    lead = phase_diagram.make_lead(**lead_pars).finalized()
    pf = phase_diagram.calculate_pfaffian(lead, params)
    gap = phase_diagram.gap_from_modes(lead, params)
    return pf * gap


fnames = [
#     "phase_diagram_gap.pickle",
#     "phase_diagram_gap_no_orbital.pickle",
#     "phase_diagram_gap_sc_inside.pickle",
    "phase_diagram_gap_sc_inside_no_orbital.pickle",
]
loss = adaptive.learner.learnerND.curvature_loss_function()

learners = []
for sc_inside_wire, orbital, Delta in (
#     [False, True, 110],
#     [False, False, 110],
#     [True, True, 0.25],
    [True, False, 0.25],
):
    f = partial(
        smallest_gap,
        params=dict(params, orbital=orbital, Delta=Delta),
        lead_pars=dict(
            lead_pars, sc_inside_wire=sc_inside_wire, with_shell=(not sc_inside_wire)
        ),
    )
    learners.append(adaptive.Learner2D(f, bounds=[(0, 2), (0, 35)]))
learner = adaptive.BalancingLearner(learners, strategy="npoints")

learner.load(fnames)

In [ ]:
from learners_file import *

In [ ]:
import adaptive

adaptive.notebook_extension()
runner = adaptive.Runner(learner, goal=lambda l: l.learners[-1].npoints > 20000)
runner.live_info()

In [ ]:
import kwant
%matplotlib inline
kwant.plot(phase_diagram.make_lead(**f.keywords['lead_pars']))

In [ ]:
runner.start_periodic_saving(dict(fname=fnames), 900)

In [ ]:
learners[0].plot(n=200, tri_alpha=0.2).Image.I[:, 6:]

In [ ]:
%%output size=100
%%opts Image [colorbar=True clim=(-0.1, 0)] 
learners[1].plot(tri_alpha=0.4).Image.I[:, 10:]

In [ ]:
runner.live_info()

In [ ]:
learners[1].npoints

In [ ]:
for l, f in zip(learners, fnames):
    l.save(f)

In [ ]:
import adaptive_scheduler

def goal(learner):
    return learner.npoints > 200

scheduler = adaptive_scheduler.scheduler.DefaultScheduler(
    cores=40,
    executor_type="ipyparallel",
)  # PBS or SLURM

run_manager = adaptive_scheduler.server_support.RunManager(
    scheduler=scheduler,
    learners_file="learners_file.py",
    goal=goal,
    log_interval=30,
    save_interval=30,
    job_name='phase-diagram'
)
run_manager.start()

In [ ]:
from learners_file import *

In [ ]:
adaptive.notebook_extension()

In [ ]:
learner = learners[0]
learner.plot(n=100)